# Data Preparation

In [1]:
#Antes de iniciar instalar
#pip3 install pydotplus
#pip3 install sklearn
#pip3 install pandas
#pip3 install numpy
#pip3 install matplotlib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [3]:
file_name='../datasets/twitter.csv'
data = pd.read_csv(file_name)
#Convertir nulos a cero
#np.nan_to_num(data)

In [4]:
#Quitando los espacios para manejo eficiente de los datos en el futuro
data.rename(columns={' Weekday':'Weekday',
 ' Hour':'Hour',
 ' Day':'Day',
 ' Lang':'Lang',
 ' IsReshare':'IsReshare',
 ' Reach':'Reach',
 ' RetweetCount':'RetweetCount',
 ' Likes':'Likes',
 ' Klout':'Klout',
 ' Sentiment':'Sentiment',
 ' text':'text',
 ' LocationID':'LocationID',
 ' UserID':'UserID' }, inplace=True)

In [5]:
data.columns

Index(['TweetID', 'Weekday', 'Hour', 'Day', 'Lang', 'IsReshare', 'Reach',
       'RetweetCount', 'Likes', 'Klout', 'Sentiment', 'text', 'LocationID',
       'UserID'],
      dtype='object')

In [6]:
df = pd.DataFrame(data)
df2 = pd.DataFrame(data)
#Eliminar todas las filas que tengan nan como valor
df.dropna(inplace = True) 
df2.dropna(inplace = True) 

In [7]:
#Información de columnas y filas, con sus respectivos nombres
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   TweetID       100000 non-null  object 
 1   Weekday       100000 non-null  object 
 2   Hour          100000 non-null  float64
 3   Day           100000 non-null  float64
 4   Lang          100000 non-null  object 
 5   IsReshare     100000 non-null  float64
 6   Reach         100000 non-null  float64
 7   RetweetCount  100000 non-null  float64
 8   Likes         100000 non-null  float64
 9   Klout         100000 non-null  float64
 10  Sentiment     100000 non-null  float64
 11  text          100000 non-null  object 
 12  LocationID    100000 non-null  float64
 13  UserID        100000 non-null  object 
dtypes: float64(9), object(5)
memory usage: 11.4+ MB
None


In [8]:
df.head()

,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,LocationID,UserID
0,tw-682712873332805633,Thursday,17.0,31.0,en,0.0,44.0,0.0,0.0,35.0,0.0,We are hiring: Senior Software Engineer - Prot...,3751.0,tw-40932430
1,tw-682713045357998080,Thursday,17.0,31.0,en,1.0,1810.0,5.0,0.0,53.0,2.0,RT @CodeMineStatus: This is true Amazon Web Se...,3989.0,tw-3179389829
2,tw-682713219375476736,Thursday,17.0,31.0,en,0.0,282.0,0.0,0.0,47.0,0.0,Devops Engineer Aws Ansible Cassandra Mysql Ub...,3741.0,tw-4624808414
3,tw-682713436967579648,Thursday,17.0,31.0,en,0.0,2087.0,4.0,0.0,53.0,0.0,Happy New Year to all those AWS instances of o...,3753.0,tw-356447127
4,tw-682714048199311366,Thursday,17.0,31.0,en,0.0,953.0,0.0,0.0,47.0,0.0,Amazon is hiring! #Sr. #International Tax Mana...,3751.0,tw-3172686669


In [9]:
df.describe()

,Hour,Day,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,LocationID
count,100000.000000,100000.000000,100000.00000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,11.412490,15.894960,0.35462,8.542396e+03,8.052750,0.152770,40.389260,0.380921,2836.163440
std,6.053577,8.399852,0.47840,8.867027e+04,97.863474,2.583633,13.636513,1.046559,1323.140242
min,0.000000,1.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000,-6.000000,1.000000
25%,7.000000,9.000000,0.00000,1.510000e+02,0.000000,0.000000,32.000000,0.000000,1601.000000
50%,11.000000,16.000000,0.00000,4.485000e+02,0.000000,0.000000,43.000000,0.000000,3738.000000
75%,16.000000,23.000000,1.00000,1.496000e+03,3.000000,0.000000,49.000000,0.666667,3775.000000
max,23.000000,31.000000,1.00000,1.034245e+07,26127.000000,133.000000,99.000000,7.333333,6289.000000


In [10]:
#Elimando una columna que no tendra uso.
del df['Weekday']
del df2['Weekday']

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100001 entries, 0 to 100000
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   TweetID       100000 non-null  object 
 1   Weekday       100000 non-null  object 
 2   Hour          100000 non-null  float64
 3   Day           100000 non-null  float64
 4   Lang          100000 non-null  object 
 5   IsReshare     100000 non-null  float64
 6   Reach         100000 non-null  float64
 7   RetweetCount  100000 non-null  float64
 8   Likes         100000 non-null  float64
 9   Klout         100000 non-null  float64
 10  Sentiment     100000 non-null  float64
 11  text          100000 non-null  object 
 12  LocationID    100000 non-null  float64
 13  UserID        100000 non-null  object 
dtypes: float64(9), object(5)
memory usage: 10.7+ MB


In [12]:
data.shape

(100001, 14)

In [13]:
#No haremos analytica del lenguaje.
del df['Lang']
del df['text']
del df2['Lang']
del df2['text']

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100001 entries, 0 to 100000
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   TweetID       100000 non-null  object 
 1   Weekday       100000 non-null  object 
 2   Hour          100000 non-null  float64
 3   Day           100000 non-null  float64
 4   Lang          100000 non-null  object 
 5   IsReshare     100000 non-null  float64
 6   Reach         100000 non-null  float64
 7   RetweetCount  100000 non-null  float64
 8   Likes         100000 non-null  float64
 9   Klout         100000 non-null  float64
 10  Sentiment     100000 non-null  float64
 11  text          100000 non-null  object 
 12  LocationID    100000 non-null  float64
 13  UserID        100000 non-null  object 
dtypes: float64(9), object(5)
memory usage: 10.7+ MB


In [15]:
data.shape

(100001, 14)

In [16]:
df.describe()

,Hour,Day,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,LocationID
count,100000.000000,100000.000000,100000.00000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,11.412490,15.894960,0.35462,8.542396e+03,8.052750,0.152770,40.389260,0.380921,2836.163440
std,6.053577,8.399852,0.47840,8.867027e+04,97.863474,2.583633,13.636513,1.046559,1323.140242
min,0.000000,1.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000,-6.000000,1.000000
25%,7.000000,9.000000,0.00000,1.510000e+02,0.000000,0.000000,32.000000,0.000000,1601.000000
50%,11.000000,16.000000,0.00000,4.485000e+02,0.000000,0.000000,43.000000,0.000000,3738.000000
75%,16.000000,23.000000,1.00000,1.496000e+03,3.000000,0.000000,49.000000,0.666667,3775.000000
max,23.000000,31.000000,1.00000,1.034245e+07,26127.000000,133.000000,99.000000,7.333333,6289.000000


In [17]:
#Normalizando los likes y reach, convirtiendolos a porcentaje entre 0 y 100% en una nueva columna
maximoL = 133
maximoR = 10342452
for i in range (0, len(df)):
    df.loc[i, 'Likes_porcentaje'] = (df.loc[i, 'Likes']/maximoL)*100
    df.loc[i, 'Reach_procentaje'] = (df.loc[i, 'Reach']/maximoR)*100
    #df2.loc[i, 'Likes_porcentaje'] = (df2.loc[i, 'Likes']/maximoL)*100
    #df2.loc[i, 'Reach_procentaje'] = (df2.loc[i, 'Reach']/maximoR)*100

In [18]:
df.describe()

,Hour,Day,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,LocationID,Likes_porcentaje,Reach_procentaje
count,100000.000000,100000.000000,100000.00000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,11.412490,15.894960,0.35462,8.542396e+03,8.052750,0.152770,40.389260,0.380921,2836.163440,0.114865,0.082595
std,6.053577,8.399852,0.47840,8.867027e+04,97.863474,2.583633,13.636513,1.046559,1323.140242,1.942581,0.857343
min,0.000000,1.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000,-6.000000,1.000000,0.000000,0.000000
25%,7.000000,9.000000,0.00000,1.510000e+02,0.000000,0.000000,32.000000,0.000000,1601.000000,0.000000,0.001460
50%,11.000000,16.000000,0.00000,4.485000e+02,0.000000,0.000000,43.000000,0.000000,3738.000000,0.000000,0.004336
75%,16.000000,23.000000,1.00000,1.496000e+03,3.000000,0.000000,49.000000,0.666667,3775.000000,0.000000,0.014465
max,23.000000,31.000000,1.00000,1.034245e+07,26127.000000,133.000000,99.000000,7.333333,6289.000000,100.000000,100.000000


In [19]:
df.head()

,TweetID,Hour,Day,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,LocationID,UserID,Likes_porcentaje,Reach_procentaje
0,tw-682712873332805633,17.0,31.0,0.0,44.0,0.0,0.0,35.0,0.0,3751.0,tw-40932430,0.0,0.000425
1,tw-682713045357998080,17.0,31.0,1.0,1810.0,5.0,0.0,53.0,2.0,3989.0,tw-3179389829,0.0,0.017501
2,tw-682713219375476736,17.0,31.0,0.0,282.0,0.0,0.0,47.0,0.0,3741.0,tw-4624808414,0.0,0.002727
3,tw-682713436967579648,17.0,31.0,0.0,2087.0,4.0,0.0,53.0,0.0,3753.0,tw-356447127,0.0,0.020179
4,tw-682714048199311366,17.0,31.0,0.0,953.0,0.0,0.0,47.0,0.0,3751.0,tw-3172686669,0.0,0.009214


In [20]:
#Data2 sera usada para añadir el país a los datos originales.
location_name ='../datasets/location.csv'
data2 = pd.read_csv(location_name)
df3 = pd.DataFrame(data2)
df3.head()

,LocationID,Country,State,StateCode,City
0,1.0,Albania,Elbasan,AL,Elbasan
1,2.0,Albania,Tirane,AL,Tirana
2,3.0,Algeria,Souk Ahras,DZ,Souk Ahras
3,4.0,Algeria,Alger,DZ,Algiers
4,5.0,Algeria,Constantine,DZ,Constantine


In [21]:
list(df3)

['LocationID', 'Country', 'State', 'StateCode', 'City']

In [22]:
#Añadir país
#for i in range (0, len(df)):
#    fila = df.loc[i, 'LocationID']
#    for j in range (0,len(df2)):
#        if fila == df2.loc[j, 'LocationID']:
#            df.loc[i, 'Country'] = df2.loc[j, 'Country']
#        else:
#            df.loc[i, 'Country']=0

In [23]:
list(df)

['TweetID',
 'Hour',
 'Day',
 'IsReshare',
 'Reach',
 'RetweetCount',
 'Likes',
 'Klout',
 'Sentiment',
 'LocationID',
 'UserID',
 'Likes_porcentaje',
 'Reach_procentaje']

In [24]:
#Eliminamos los datos que no se usaran para el Reto3
del df['TweetID']
del df['Likes']
del df['UserID']
del df['LocationID']
del df['Sentiment']
del df['Reach']
del df['Likes_porcentaje']
#Eliminamos los datos no se usaran para el Reto3 opción 2
del df2['TweetID']
del df2['Klout']
del df2['Sentiment']
del df2['LocationID']
del df2['UserID']

In [26]:
list(df)

['Hour', 'Day', 'IsReshare', 'RetweetCount', 'Klout', 'Reach_procentaje']

In [27]:
list(df2)

['Hour', 'Day', 'IsReshare', 'Reach', 'RetweetCount', 'Likes']

In [28]:
#Cambiando el orden, para que klout quede como la columna de clasificación
columnsTitles=['Hour', 'Day', 'Reach_procentaje', 'IsReshare', 'RetweetCount', 'Klout']
df=df.reindex(columns=columnsTitles)

In [29]:
df.describe()

,Hour,Day,Reach_procentaje,IsReshare,RetweetCount,Klout
count,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000
mean,11.412490,15.894960,0.082595,0.35462,8.052750,40.389260
std,6.053577,8.399852,0.857343,0.47840,97.863474,13.636513
min,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000
25%,7.000000,9.000000,0.001460,0.00000,0.000000,32.000000
50%,11.000000,16.000000,0.004336,0.00000,0.000000,43.000000
75%,16.000000,23.000000,0.014465,1.00000,3.000000,49.000000
max,23.000000,31.000000,100.000000,1.00000,26127.000000,99.000000


In [30]:
list(df)

['Hour', 'Day', 'Reach_procentaje', 'IsReshare', 'RetweetCount', 'Klout']

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Hour              100000 non-null  float64
 1   Day               100000 non-null  float64
 2   Reach_procentaje  100000 non-null  float64
 3   IsReshare         100000 non-null  float64
 4   RetweetCount      100000 non-null  float64
 5   Klout             100000 non-null  float64
dtypes: float64(6)
memory usage: 7.8 MB


In [33]:
df = df.astype({"Hour":'float32', "Day":'float32', "Reach_procentaje":'float32', "IsReshare":'float32', "RetweetCount":'float32', "Klout":'float32'})
df2 = df2.astype({"Hour":'float32', "Day":'float32', "IsReshare":'float32', "Reach":'float32', "RetweetCount":'float32', "Likes":'float32'}) 

In [ ]:
#df.astype('float32').dtypes

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Hour              100000 non-null  float32
 1   Day               100000 non-null  float32
 2   Reach_procentaje  100000 non-null  float32
 3   IsReshare         100000 non-null  float32
 4   RetweetCount      100000 non-null  float32
 5   Klout             100000 non-null  float32
dtypes: float32(6)
memory usage: 5.6 MB


In [35]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Hour          100000 non-null  float32
 1   Day           100000 non-null  float32
 2   IsReshare     100000 non-null  float32
 3   Reach         100000 non-null  float32
 4   RetweetCount  100000 non-null  float32
 5   Likes         100000 non-null  float32
dtypes: float32(6)
memory usage: 3.1 MB


In [36]:
df.to_csv('../datasets/datosLimpios.csv', float_format="%.2f")

In [37]:
df2.to_csv('../datasets/datosLimpios2.csv', float_format="%.2f")